In [ ]:
# Initialize drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Move to Google Drive 
%cd drive
%cd 'My Drive'
%cd 'MSc Stats Dissertation'

/content/drive
/content/drive/My Drive
/content/drive/My Drive/MSc Stats Dissertation


In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
!pip install selfies
!pip install deepsmiles

--2020-07-26 20:23:54--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-07-26 20:23:54--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
expected: 6e267728e74a7089e59c7d8c59d20e6b
     got: ebb76490dfe6ce63b1dec5392631d80d  -
Unpacking payload ...
Solving envi

In [ ]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras as keras
import pandas as pd
import math
import tensorflow.keras.layers as layers
from selfies import encoder, decoder
import deepsmiles
import rdkit
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Import Necessary Data for training
train_smiles_X = np.load('./vocab/train_selfies_X.npy',allow_pickle=True)
vocab =np.load('./vocab/selfies_vocab.npy',allow_pickle=True)
vocab_index = np.load('./vocab/selfies_vocab_index.npy',allow_pickle=True)
vocab = dict(vocab.ravel()[0])
vocab_index = dict(vocab_index.ravel()[0])
gene_expressions = np.load('./Datasets/ordered_gene_expressions.npy')
ic50 = np.load('./Datasets/ordered_ic50.npy')
smiles_pairs = np.load('./Datasets/ordered_smiles.npy')


In [ ]:
## Neccesary CONSTANTS
BATCH_SIZE = 256
VOCAB_SIZE = len(vocab_index)
EPOCHS = 30
LEARNING_RATE = 1e-4
DROP_OUT= 0.2
EMBEDDING_DIM = 192  ## Embedding dim of the characters
LATENT_DIM = 64
PAD_LEN = 250
MAX_LEN = PAD_LEN -1
DROPOUT = 0.2
TRAIN = True

In [ ]:
## Get randomized chemistry of different smiles strings
## for training
from rdkit import Chem
def randomlabels(mol, N):
  ans = set()
  mol_ex = None
  if mol.find('.') != -1:
    mol_ex = mol[mol.find('.'):]
    mol = mol[:mol.find('.')]
  molt = Chem.MolFromSmiles(mol)
  while len(ans) < N:
    if mol_ex == None:
      ans.add(Chem.MolToSmiles(molt,doRandom=True,canonical=True))
    else:
      ans.add(Chem.MolToSmiles((molt),doRandom=True,canonical=True)+mol_ex)
  return sorted(list(ans))

In [ ]:
## converts smiles to seflies smiles 
def get_smiles_from_selfies(selfies_list):
  smiles = []
  for selfie in selfies_list:
    try:
      smile = decoder(selfie)
    except:
      smile = None
    smiles.append(smile)
  return smiles

In [ ]:
def integer_encode_selfies(selfies,vocab_dict):
  selfies_enc = []
  for char in selfies:
    selfies_enc.append(vocab_dict[char])
  return selfies_enc

In [ ]:
## Splits the selfies <molecule> into a list of character strings.
def split_selfie(molecule):
  return re.findall(r'\[.*?\]|\.', molecule)

In [ ]:
## Takes processed selfies smiles and returns the tokenized 
## versions of the selfies
def tokenize_selfies(selfies):
  char_list = split_selfie(selfies)
  tokenized= []
  tokenized.append('<BOS>')
  i = 0 
  while i < len(char_list):
    char = char_list[i]
    tokenized.append(char)
    i = i+1
  tokenized.append('<EOS>')
  return tokenized

In [ ]:
import re
## replace Br and Cl with single letters
def replace_halogens(string):
  br = re.compile('Br')
  cl = re.compile('Cl')
  string = br.sub('R', string)
  string = cl.sub('L', string)
  return string

In [ ]:
### Properly scale data 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(ic50.reshape(-1,1))
ic50 = scaler.transform(ic50.reshape(-1,1))
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(gene_expressions)
gene_expressions = scaler.transform(gene_expressions)

In [ ]:
new_gene_expressions = []
new_ic50 = []
new_smile_pairs = []

In [ ]:
## Get training data for guided training 
for i in range(len(smiles_pairs)):
  if i % 10000 == 0:
    print(i)
  random_mols = randomlabels(smiles_pairs[i],8)
  if SELFIES:
    for mols in random_mols:
      new_smile_pairs.append(encoder(mols))
  else:
    new_smile_pairs.extend(random_mols)
  for j in range(len(random_mols)):
    new_gene_expressions.append(gene_expressions[i])
    new_ic50.append(ic50[i])
gene_expressions= new_gene_expressions 
ic50 = new_ic50 
smiles_pairs = new_smile_pairs

In [ ]:
smile_pair_tokens = []
for smiles in smiles_pairs:
  if SELFIES:
    smile_pair_tokens.append(tokenize_selfies(smiles))
  else:
    smiles = replace_halogens(smiles)
    smile_pair_tokens.append(tokenize_smiles(smiles))
smile_pair_tokens = np.array(smile_pair_tokens)

In [ ]:
smiles_ordered = []
for smiles in smile_pair_tokens:
  if SELFIES:
    smiles_ordered.append(integer_encode_selfies(smiles,vocab))
  else:
    smiles_ordered.append(integer_encode(smiles,vocab))
smiles_ordered = np.array(smiles_ordered)

In [ ]:
t = tf.keras.preprocessing.sequence.pad_sequences(smiles_ordered,maxlen=PAD_LEN,padding='post')
NUM_BATCHES = math.floor(len(t)/BATCH_SIZE )

In [ ]:
NUM_TRAIN_BATCH = math.floor(NUM_BATCHES*0.99)
NUM_TEST_BATCH = math.floor(NUM_BATCHES*(0.01))

In [ ]:
test_smiles = t[NUM_TRAIN_BATCH*BATCH_SIZE:(NUM_TEST_BATCH+NUM_TRAIN_BATCH)*BATCH_SIZE]
train_smiles = t[:NUM_TRAIN_BATCH*BATCH_SIZE]

test_genes = gene_expressions[NUM_TRAIN_BATCH*BATCH_SIZE:(NUM_TEST_BATCH+NUM_TRAIN_BATCH)*BATCH_SIZE]
train_genes = gene_expressions[:NUM_TRAIN_BATCH*BATCH_SIZE]

test_ic50 = ic50[NUM_TRAIN_BATCH*BATCH_SIZE:(NUM_TEST_BATCH+NUM_TRAIN_BATCH)*BATCH_SIZE]
train_ic50 = ic50[:NUM_TRAIN_BATCH*BATCH_SIZE]


In [ ]:
import GANS.ic50vae as conv_smiles_vae
smile_vae = conv_smiles_vae.SMILE_VAE(vocab_size= VOCAB_SIZE,embedding_dim=EMBEDDING_DIM, max_len= MAX_LEN, 
                     latent_dim = LATENT_DIM, recurrent_dropout = DROP_OUT,dropout_rate= DROP_OUT)
if SELFIES:
  smile_vae.load_weights('ic50g_selfies_conv_vae_weights')
else:
  smile_vae.load_weights('smiles_conv_vae_weights2')

In [ ]:
mca_ic50 = ic50mca.IC50_MCA(vocab_size=VOCAB_SIZE, 
                    embedding_dim =EMBEDDING_DIM, num_genes =2128,
                    hidden_dim = HIDDEN_DIM, max_len = MAX_LEN, 
                    latent_dim = LATENT_DIM)

In [ ]:
def train_smile_gene_ca(mca_ic50,smile_vae,
                        train_smiles,train_genes,train_ic50,
                        test_smiles, test_genes,test_ic50):
  clip = -1
  display_step = 100
  STEPS_PER_EPOCH = train_smiles.shape[0]//BATCH_SIZE
  TEST_STEPS = test_smiles.shape[0]//BATCH_SIZE
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-6)
  
  for epoch in range(EPOCHS):
    ## Randomize the training process
    indxs = np.arange(STEPS_PER_EPOCH)
    np.random.shuffle(indxs)
    total_train_loss = 0
    for batch in range(STEPS_PER_EPOCH):
      with tf.GradientTape() as tape:

        ## Get relevant batch data
        train_smiles_batch = np.array(train_smiles[indxs[batch]*BATCH_SIZE: indxs[batch]*BATCH_SIZE + BATCH_SIZE])
        train_gene_batch = np.array(train_genes[indxs[batch]*BATCH_SIZE: indxs[batch]*BATCH_SIZE + BATCH_SIZE])
        train_ic50_batch = np.array(train_ic50[indxs[batch]*BATCH_SIZE: indxs[batch]*BATCH_SIZE + BATCH_SIZE])
        ## Get logits of the model 
        h, z_mean,z_log_var = smile_vae.encoder(train_smiles_batch[:,:-1])
        z = tf.keras.layers.Lambda(smile_vae.encoder.sample, 
                               output_shape =(LATENT_DIM,))([z_mean,z_log_var])

        ## Get ic50 prediction              
        ic50_pred = mca_ic50(encoded_smiles = z_mean, genes = train_gene_batch)
        loss_op = tf.reduce_sum(tf.keras.losses.MSE(train_ic50_batch,ic50_pred))
        total_train_loss += loss_op
        
      ## Apply gradients 
      gradients = tape.gradient(loss_op, mca_ic50.trainable_variables)
      if clip != -1:
        gradients, _ = tf.clip_by_global_norm(gradients, clip) 
      optimizer.apply_gradients(zip(gradients, mca_ic50.trainable_variables))

      ## Diplay training loss information ##
      if batch % display_step == 0 or batch == 1:

        ### Get logits for test data
        rand_int =np.random.randint(low=0,high = TEST_STEPS)
        test_smiles_batch = np.array(test_smiles[rand_int*BATCH_SIZE: rand_int*BATCH_SIZE + BATCH_SIZE])
        test_gene_batch = np.array(test_genes[rand_int*BATCH_SIZE: rand_int*BATCH_SIZE + BATCH_SIZE])
        test_ic50_batch = np.array(test_ic50[rand_int*BATCH_SIZE: rand_int*BATCH_SIZE + BATCH_SIZE])

        ### Encode data using encoder 
        h, z_mean,z_log_var = smile_vae.encoder(test_smiles_batch[:,:-1])
        z = tf.keras.layers.Lambda(smile_vae.encoder.sample, 
                               output_shape =(LATENT_DIM,))([z_mean,z_log_var])

        ## Get prediction  
        ic50_pred_test = mca_ic50(encoded_smiles = z_mean,
                             genes = test_gene_batch,training=False)

        ### Print out test accuracy on model 
        test_loss = tf.reduce_sum(tf.keras.losses.MSE(test_ic50_batch,
                                                     ic50_pred_test))

        print("Step " + str(batch) + ", Training Loss = " + \
                "{:.3f}".format(tf.reduce_mean(loss_op)) + ", Test Loss = " + \
                "{:.3f}".format(tf.reduce_mean(test_loss)))
      if  (batch) == STEPS_PER_EPOCH-1:
        print("TOTAL TRAINING LOSS " + "{:.3f}".format(tf.reduce_sum(total_train_loss)))
      if  (batch) % 3000 == 0:
        mca_ic50.save_weights('ic50network_selfies_ic50g')

In [ ]:
if TRAIN:
  train_smile_gene_ca(mca_ic50,smile_vae,
                        train_smiles,train_genes,train_ic50,
                        test_smiles, test_genes,test_ic50)
else:
  mca_ic50.load_weights('ic50network_selfies_ic50g')

HERE
Step 0, Training Loss = 94.815, Test Loss = 128.445
Step 1, Training Loss = 152.783, Test Loss = 108.657
Step 100, Training Loss = 14.438, Test Loss = 8.992
Step 200, Training Loss = 7.997, Test Loss = 7.166
Step 300, Training Loss = 10.095, Test Loss = 6.222
Step 400, Training Loss = 5.869, Test Loss = 5.126
Step 500, Training Loss = 5.062, Test Loss = 5.482
Step 600, Training Loss = 6.577, Test Loss = 5.190
Step 700, Training Loss = 4.620, Test Loss = 4.896
Step 800, Training Loss = 5.242, Test Loss = 5.035
Step 900, Training Loss = 5.991, Test Loss = 4.571
Step 1000, Training Loss = 6.230, Test Loss = 4.978
Step 1100, Training Loss = 2.892, Test Loss = 4.632
Step 1200, Training Loss = 3.160, Test Loss = 4.239
Step 1300, Training Loss = 4.498, Test Loss = 5.152
Step 1400, Training Loss = 4.295, Test Loss = 4.759
Step 1500, Training Loss = 3.999, Test Loss = 3.943
Step 1600, Training Loss = 4.376, Test Loss = 4.527
Step 1700, Training Loss = 3.281, Test Loss = 4.465
Step 1800, Tr

In [ ]:
mca_ic50.summary()

Model: "i_c50_mca"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector_1 (RepeatVecto multiple                  0         
_________________________________________________________________
contextual_attention_layer ( multiple                  609281    
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  9359360   
_________________________________________________________________
dense_attention_layer (Dense multiple                  4530512   
_________________________________________________________________
dense_12 (Dense)             multiple                  20